In [ ]:
from datascience import *
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
def draw_and_compare(true_slope, true_int, sample_size):
    x = np.random.normal(50, 5, sample_size)
    xlims = np.array([np.min(x), np.max(x)])
    errors = np.random.normal(0, 6, sample_size)
    y = (true_slope * x + true_int) + errors
    sample = Table().with_columns('x', x, 'y', y)

    sample.scatter('x', 'y')
    plots.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plots.title('True Line, and Points Created')

    sample.scatter('x', 'y')
    plots.title('What We Get to See')

    sample.scatter('x', 'y', fit_line=True)
    plots.title('Regression Line: Estimate of True Line')

    sample.scatter('x', 'y', fit_line=True)
    plots.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plots.title("Regression Line and True Line")

## Linear regression

In [ ]:
def standard_units(arr):
    return (arr - np.average(arr))/np.std(arr)

def correlation(t, x, y):
    x_standard = standard_units(t.column(x))
    y_standard = standard_units(t.column(y))
    return np.average(x_standard * y_standard)

def slope(t, x, y):
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def intercept(t, x, y):
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

In [ ]:
def prediction_at(t, x, y, x_value):
    '''
    t - table
    x - label of x column
    y - label of y column
    x_value - the x value for which we want to predict y
    '''
    return slope(t, x, y) * x_value + intercept(t, x, y)

## Regression Model  - Sample vs Population Lines ##

In [ ]:
draw_and_compare(-2, 5, 50)

## Prediction ##

In [ ]:
births = Table.read_table('baby.csv')
births.show(3)

In [ ]:
births = births.where('Gestational Days', are.between(240, 320))

In [ ]:
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)

In [ ]:
prediction_at_300 = prediction_at(births, 'Gestational Days', 'Birth Weight', 300)
prediction_at_300

## Confidence Interval for Prediction ##

In [ ]:
def bootstrap_prediction(t, x, y, new_x, repetitions=1000):

    # Bootstrap the scatter, predict, collect
    predictions = make_array()
    for i in np.arange(repetitions):
        resample = t.sample()
        predicted_y = prediction_at(resample, x, y, new_x)
        predictions = np.append(predictions, predicted_y)

    # Find the ends of the approximate 95% prediction interval
    left = percentile(2.5, predictions)
    right = percentile(97.5, predictions)

    # Display results
    Table().with_column('Prediction', predictions).hist(bins=20)
    plots.xlabel('predictions at x='+str(new_x))
    plots.plot([left, right], [0, 0], color='yellow', lw=8);
    print('Approximate 95%-confidence interval for height of true line:')
    print(left, right, '(width =', right - left, ')') 

In [ ]:
bootstrap_prediction(births, 'Gestational Days', 'Birth Weight', 300)

## Inference for the Slope ##

In [ ]:
births.scatter('Gestational Days', 'Birth Weight', fit_line=True)

In [ ]:
slope(births, 'Gestational Days', 'Birth Weight')

In [ ]:
def bootstrap_slope(t, x, y, repetitions=1000):
    
    # Bootstrap the scatter, find the slope, collect
    slopes = make_array()
    for i in np.arange(repetitions):
        bootstrap_sample = t.sample()
        bootstrap_slope = slope(bootstrap_sample, x, y)
        slopes = np.append(slopes, bootstrap_slope)
    
    # Find the endpoints of the 95% confidence interval for the true slope
    left = percentile(2.5, slopes)
    right = percentile(97.5, slopes)
    
    # Slope of the regression line from the original sample
    observed_slope = slope(t, x, y)
    
    # Display results
    Table().with_column('Bootstrap Slopes', slopes).hist(bins=20)
    plots.plot(make_array(left, right), make_array(0, 0), color='yellow', lw=8);
    print('Slope of regression line:', observed_slope)
    print('Approximate 95%-confidence interval for the slope of the true line:')
    print(left, 'to', right)

In [ ]:
bootstrap_slope(births, 'Gestational Days', 'Birth Weight')

## Rain on the Regression Parade

In [ ]:
draw_and_compare(0, 10, 25)

**Null Hypothesis.** Slope of true line = 0.

**Alternative Hypothesis.** Slope of true line is not 0.

In [ ]:
slope(births, 'Maternal Age', 'Birth Weight')

In [ ]:
births.scatter('Maternal Age', 'Birth Weight', fit_line=True)

In [ ]:
bootstrap_slope(births, 'Maternal Age', 'Birth Weight', 1000)

## Multiple Regression

In [ ]:
births = Table.read_table('baby.csv')
births.show(3)

In [ ]:
correlation(births, 'Birth Weight', 'Gestational Days')

In [ ]:
correlation(births, 'Birth Weight', 'Maternal Age')

In [ ]:
correlation(births, 'Birth Weight', 'Maternal Height')

In [ ]:
correlation(births, 'Birth Weight', 'Maternal Pregnancy Weight')

What if we try to predict birth weight from gestational days, maternal height, and maternal pregnancy weight?

In [ ]:
def multiple_regression_rmse(gest_days_mult, mat_height_mult, mat_preg_weight_mult, intercept):
    
    gest_days = births.column('Gestational Days')
    mat_height = births.column('Maternal Height')
    mat_preg_weight = births.column('Maternal Pregnancy Weight')
    
    y = births.column('Birth Weight')
    
    prediction = sum([
        gest_days_mult * gest_days,
        mat_height_mult * mat_height,
        mat_preg_weight_mult * mat_preg_weight,
        intercept]
    )
    
    mse = np.mean((y - prediction) ** 2)
    return np.sqrt(mse)

In [ ]:
best = minimize(multiple_regression_rmse)
best

This means that our prediction for birth weight is:

0.3707216(*number of gestational days*) + 0.09485383(*Maternal Height*) + 0.10960043(*Maternal Pregnancy Weight*) + -4.16139904

In [ ]:
mult_predictions = sum([
    best.item(0) * births.column('Gestational Days'),
    best.item(1) * births.column('Maternal Height'),
    best.item(2) * births.column('Maternal Pregnancy Weight'),
    best.item(3)]
)

mult_resid = births.column('Birth Weight') - mult_predictions 

In [ ]:
births = births.with_columns(
    'Mult Predictions', mult_predictions,
    'Mult Residuals', mult_resid 
)

In [ ]:
births.scatter('Mult Predictions', 'Mult Residuals')

In [ ]:
births.where('Mult Predictions', are.between(90, 150)).scatter('Mult Predictions', 'Mult Residuals')

In [ ]:
births.scatter('Maternal Height', 'Mult Residuals')

In [ ]:
births.scatter('Maternal Pregnancy Weight', 'Mult Residuals')

In [ ]:
births.scatter('Gestational Days', 'Mult Residuals')

In [ ]:
births.where('Gestational Days', are.between(210, 360)).scatter('Gestational Days', 'Mult Residuals')